In [1]:
from __future__ import division, absolute_import, print_function

import sys
import os
import pickle
import numpy as np
import random

import h5py
from sklearn import metrics 
from collections import Counter
from glob import glob
import gc
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tables import *

from keras import backend as K 
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.backend import manual_variable_initialization 
from keras.models import load_model, Model
from keras.layers import Dense, concatenate, Flatten, Conv1D, BatchNormalization, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, TerminateOnNaN
#import keras

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)


from src.model_functions import *
from src.Target import Target
from src.postproc_auxiliar_functions import *

os.environ['PYTHONHASHSEED'] = '0' 
np.random.seed(8)
random.seed(8)
tf.random.set_seed(8)

Using TensorFlow backend.


In [2]:
absPath

'/home/angela3/imbalance_pcm_benchmark/'

In [3]:
nfolds = 10
batch_size = 128
epochss = 100
type_padding_prot = "pre_padding"
protein_type = "GPCRs" #"kinases"

In [4]:
#Opening HDF5 with data
file_h5 = "".join((absPath, "data/", protein_type,"/no_resampling/compounds_activity.h5"))
f = h5py.File(file_h5, 'r')
group = '/activity'
table = "prot_comp"

#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()

In [5]:
max_len_prot

1499

In [6]:
learning_rate = 5e-4
decay_rate = learning_rate/epochss
adamm = Adam(lr=learning_rate, beta_1=0.1, beta_2=0.001, epsilon=1e-08, decay=decay_rate)

In [7]:
# LEFT BLOCK (to analyse amino acid sequences)
input_seq = Input(shape=(max_len_prot, len(prot_dict)), dtype='float32')
conv_seq = Conv1D(filters=64, padding='same', strides=1, kernel_size=3, activation='relu')(input_seq)
dropout_1 = Dropout(0.4)(conv_seq)
flatten_seq = Flatten()(dropout_1)#(dense_seq)
dense_seq_2 = Dense(50)(flatten_seq)
dropout_2 = Dropout(0.4)(dense_seq_2)

#RIGHT BRANCH (to analyse fingerprints)
input_fps = Input(shape=(881,), dtype='float32')
dense_fps = Dense(50)(input_fps)
dropout_3 = Dropout(0.4)(dense_fps)
#bn_3 =  BatchNormalization()(dense_fps)#(dense_seq_2)#(conv_seq)


#MERGE BOTH BRANCHES
main_merged = concatenate([dropout_2, dropout_3],axis=1)#([dense_seq_2, dense_fps], axis=1)

main_dense = Dense(2, activation='softmax')(main_merged)

#build and compile model
model = Model(inputs=[input_seq, input_fps], outputs=[main_dense])
model.compile(loss='categorical_crossentropy', optimizer = adamm, metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1499, 26)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1499, 64)     5056        input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1499, 64)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 95936)        0           dropout_1[0][0]                  
____________________________________________________________________________________________

In [ ]:
for fold in range(nfolds):
#for fold in range(1):
    print("Fold:", str(fold))
    file_list = "".join((absPath, "data/", protein_type, "/no_resampling/splitting_lists/splitting_",
                               str(fold), "_list.pickle"))
    with open(file_list, "rb") as input_file:
        splitting_list = pickle.load(input_file)    
    
    splitting_list[0].sort()
    splitting_list[1].sort()
    #Defining generators
    train_generator = batch_generator_DL(batch_size, f, group, table, splitting_list[0], 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    val_generator = batch_generator_DL(batch_size, f, group, table, splitting_list[1], 
                                     max_len_prot, type_padding_prot=type_padding_prot)
    
    #defining callbacks
    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/no_resampling/logs/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/no_resampling/logs/", str(fold), "/")))
    
    log_path = "".join((absPath, "data/", protein_type, 
                                   "/no_resampling/logs/", str(fold), "/training_log.csv"))
    csv_logger = CSVLogger(log_path)

    if not os.path.exists("".join((absPath, "data/", protein_type, 
                                   "/no_resampling/checkpoint/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, 
                                   "/no_resampling/checkpoint/", str(fold), "/")))

    #if there are already files in the folder, it removes them
    r = glob("".join((absPath, "data/", protein_type, 
                                   "/no_resampling/checkpoint/", str(fold), "/*")))
    for i in r:
        os.remove(i)
   
    terminan = TerminateOnNaN()
    checkpoint_path = "".join((absPath, "data/", protein_type, 
                                   "/no_resampling/checkpoint/", str(fold),
                               "/weights-improvement-{epoch:03d}-{val_accuracy:.4f}.hdf5"))
    mcheckpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=0, 
                                          save_best_only=True, save_weights_only=False)

    callbacks_list = [csv_logger, terminan, mcheckpoint ]
    print("Training")
    # fitting the model
    history = model.fit_generator(generator=train_generator, 
                              validation_data=val_generator,
                             steps_per_epoch= int(len(splitting_list[0])/batch_size),
                              validation_steps=int(len(splitting_list[1])/batch_size),
                             epochs=epochss,
                             callbacks=callbacks_list,
                             verbose=1)
    #saving history
    if not os.path.exists("".join((absPath, "data/", protein_type, "/no_resampling/results/", 
                                   str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/no_resampling/results/", str(fold), "/")))

    with open("".join((absPath, "data/", protein_type, "/no_resampling/results/", str(fold), "/history.pickle")), 'wb') as handle:
        pickle.dump(history, handle)
        
    print("Prediction on test data")
    splitting_list[2].sort()
    #PROTEINS
    batch_sequences = list(f[group][table][splitting_list[2]]["sequence"])
    #COMPOUNDS
    batch_compounds = list(f[group][table][splitting_list[2]]["fingerprint"])
    #LABELS
    batch_y = list(f[group][table][splitting_list[2]]["label"])
    #processing sequences and compounds
    seqs_onehot = np.asarray(processing_sequences(batch_sequences, max_len_prot, type_padding_prot))
    comps_batch = np.asarray(processing_fingerprints(batch_compounds))
    batch_labels = np.asarray(bin_to_onehot(batch_y))
    
    history_path = "".join((absPath, "data/", protein_type, "/no_resampling/results/", 
                                   str(fold), "/history.pickle"))
    path_to_confusion = "".join((absPath, "data/", protein_type, "/no_resampling/results/", 
                                   str(fold), "/"))
    path_to_auc = "".join((absPath, "data/", protein_type, "/no_resampling/results/", 
                                   str(fold), "/"))
    
    history = plot_history(history_path, "".join((absPath, "data/", protein_type, 
                                                  "/no_resampling/results/", 
                                   str(fold), "/")))
    path_to_cp = ''.join((absPath, "data/", protein_type, "/no_resampling/checkpoint/", 
                          str(fold), "/"))

    model, best_path = load_best_model(history, path_to_cp)

    cps_loc = ''.join((absPath, "data/", protein_type, "/no_resampling/checkpoint/", 
                          str(fold), "/*.hdf5")) 

    #removing the rest of weights
    fileList = glob(cps_loc, recursive=True)
    fileList.remove(best_path)
    if len(fileList) >1:
        for filePath in fileList:
            try:
                os.remove(filePath)
            except OSError:
                print("Error while deleting file")
    
    y_predprob = model.predict([seqs_onehot, comps_batch])
    y_prob = y_predprob[:,1]
    y_pred = y_predprob.argmax(-1)
    y_test = batch_labels.argmax(-1)
    print("Counting predicted: ", Counter(y_pred))
    
    batch_compID_test = list(f[group][table][splitting_list[2]]["da_comp_id"])
    batch_protID_test = list(f[group][table][splitting_list[2]]["da_prot_id"])
    
    #confusion matrix
    confusion_matrix(y_test, y_pred, path_to_confusion)
        
    #AUC
    file_auc = ''.join((absPath, "data/", protein_type, "/no_resampling/results/", 
                                   str(fold), "/AUC.pickle"))
    compute_roc(y_test, y_prob, path_to_auc)
    
    # saving predictions on test set

    predictions_test = pd.DataFrame({"y_test":y_test, "y_prob":y_prob, "y_pred":y_pred, "comp_ID": batch_compID_test,
                                "DeepAffinity Protein ID": batch_protID_test})

    if not os.path.exists("".join((absPath, "data/", protein_type, "/no_resampling/predictions/", str(fold), "/"))):
        os.makedirs("".join((absPath, "data/", protein_type, "/no_resampling/predictions/", str(fold), "/")))

    predictions_test.to_csv("".join((absPath, "data/", protein_type, "/no_resampling/predictions/", str(fold), "/test.csv")))

Fold: 0
Training
Epoch 1/100
1219/1219 [==============================] - 103s 84ms/step - loss: 0.8163 - accuracy: 0.7446 - val_loss: 0.3536 - val_accuracy: 0.7528
Epoch 2/100
1219/1219 [==============================] - 94s 77ms/step - loss: 0.5094 - accuracy: 0.7755 - val_loss: 0.2929 - val_accuracy: 0.7553
Epoch 3/100
1219/1219 [==============================] - 96s 79ms/step - loss: 0.5053 - accuracy: 0.7774 - val_loss: 0.7465 - val_accuracy: 0.7607
Epoch 4/100
1219/1219 [==============================] - 95s 78ms/step - loss: 0.5025 - accuracy: 0.7785 - val_loss: 0.6235 - val_accuracy: 0.7601
Epoch 5/100
1219/1219 [==============================] - 95s 78ms/step - loss: 0.5027 - accuracy: 0.7784 - val_loss: 0.2535 - val_accuracy: 0.7625 - loss: 0.5029 - accuracy: 0.77
Epoch 6/100
1219/1219 [==============================] - 95s 78ms/step - loss: 0.5019 - accuracy: 0.7786 - val_loss: 0.5030 - val_accuracy: 0.7616
Epoch 7/100
1219/1219 [==============================] - 95s 78ms/st

1219/1219 [==============================] - 97s 80ms/step - loss: 0.4930 - accuracy: 0.7838 - val_loss: 0.6775 - val_accuracy: 0.7456
Epoch 52/100
1219/1219 [==============================] - 98s 80ms/step - loss: 0.4933 - accuracy: 0.7831 - val_loss: 0.3709 - val_accuracy: 0.7415
Epoch 53/100
1219/1219 [==============================] - 98s 81ms/step - loss: 0.4934 - accuracy: 0.7823 - val_loss: 0.4779 - val_accuracy: 0.7239
Epoch 54/100
1219/1219 [==============================] - 98s 80ms/step - loss: 0.4952 - accuracy: 0.7819 - val_loss: 0.6000 - val_accuracy: 0.7453
Epoch 55/100
1219/1219 [==============================] - 96s 79ms/step - loss: 0.4924 - accuracy: 0.7842 - val_loss: 0.2634 - val_accuracy: 0.7446
Epoch 56/100
1219/1219 [==============================] - 97s 80ms/step - loss: 0.4930 - accuracy: 0.7832 - val_loss: 0.5543 - val_accuracy: 0.7424
Epoch 57/100
1219/1219 [==============================] - 96s 79ms/step - loss: 0.4937 - accuracy: 0.7839 - val_loss: 0.1459 

1219/1219 [==============================] - 96s 79ms/step - loss: 0.4923 - accuracy: 0.7836 - val_loss: 0.5502 - val_accuracy: 0.7065
Epoch 98/100
1219/1219 [==============================] - 96s 79ms/step - loss: 0.4939 - accuracy: 0.7842 - val_loss: 0.6356 - val_accuracy: 0.7289
Epoch 99/100
1219/1219 [==============================] - 97s 80ms/step - loss: 0.4940 - accuracy: 0.7844 - val_loss: 0.9037 - val_accuracy: 0.6763TA: 30s - loss: 0.4954 - ETA: 25s - loss: 0.49 - ETA: 17s - loss: 0.4987 - accuracy: 0.779 - ETA: 17s - loss: 0.4989 - accuracy - ETA: 16s - loss: 0.4993 - accuracy: 0.7 - ETA: 16s - lo - ETA: 14s - loss: 0.49 - ETA: 1
Epoch 100/100
1219/1219 [==============================] - 98s 80ms/step - loss: 0.4956 - accuracy: 0.7828 - val_loss: 0.5813 - val_accuracy: 0.6877
Prediction on test data
best epoch:  007
Counting predicted:  Counter({1: 20256, 0: 879})

Model Report
Accuracy (test set): 0.7805
Confusion matrix:
[[  552  4313]
 [  327 15943]]
Detailed classificati

1241/1241 [==============================] - 95s 77ms/step - loss: 0.4946 - accuracy: 0.7809 - val_loss: 0.2471 - val_accuracy: 0.7852
Epoch 39/100
1241/1241 [==============================] - 95s 76ms/step - loss: 0.4936 - accuracy: 0.7808 - val_loss: 0.6369 - val_accuracy: 0.7808
Epoch 40/100
1241/1241 [==============================] - 96s 77ms/step - loss: 0.4951 - accuracy: 0.7797 - val_loss: 0.1659 - val_accuracy: 0.7795
Epoch 41/100
1241/1241 [==============================] - 95s 77ms/step - loss: 0.4928 - accuracy: 0.7811 - val_loss: 0.3171 - val_accuracy: 0.7813
Epoch 42/100
1241/1241 [==============================] - 95s 76ms/step - loss: 0.4935 - accuracy: 0.7811 - val_loss: 0.5089 - val_accuracy: 0.7789
Epoch 43/100
1241/1241 [==============================] - 96s 77ms/step - loss: 0.4948 - accuracy: 0.7804 - val_loss: 0.6533 - val_accuracy: 0.7800
Epoch 44/100
1241/1241 [==============================] - 95s 77ms/step - loss: 0.4960 - accuracy: 0.7807 - val_loss: 0.9852 

1241/1241 [==============================] - 95s 76ms/step - loss: 0.4928 - accuracy: 0.7816 - val_loss: 0.6540 - val_accuracy: 0.7805
Epoch 79/100
1241/1241 [==============================] - 94s 76ms/step - loss: 0.4910 - accuracy: 0.7831 - val_loss: 0.2175 - val_accuracy: 0.7766
Epoch 80/100
1241/1241 [==============================] - 96s 77ms/step - loss: 0.4922 - accuracy: 0.7821 - val_loss: 0.1838 - val_accuracy: 0.7742 loss: 0.4963 - accuracy:  - ETA: 21s - loss: 0.4952 - accuracy - ETA: 20s - loss: 0.495 - ETA: 19s - loss: 0.4945 - accuracy: 0.7 - ETA: 19s - loss: 0.4944 - accuracy:  - ETA: 18s - 
Epoch 81/100
1241/1241 [==============================] - 96s 77ms/step - loss: 0.4902 - accuracy: 0.7832 - val_loss: 0.2711 - val_accuracy: 0.7769 0.4818 - acc - ETA: 52s - loss: 0 - ETA: 43s - loss: - ETA: 41s - loss: 0.4905 - accurac - ETA: 40s - lo - ETA: 38s - loss: 0.4 - ETA: 36s - loss: 0.4873 - ac - ETA: 35s - loss: 0.4871 - accuracy:  - E - ETA: 1s - loss:
Epoch 82/100
1241/

...

1230/1230 [==============================] - 100s 81ms/step - loss: 0.4863 - accuracy: 0.7858 - val_loss: 0.8741 - val_accuracy: 0.7310
Epoch 69/100
1230/1230 [==============================] - 101s 82ms/step - loss: 0.4881 - accuracy: 0.7856 - val_loss: 1.0018 - val_accuracy: 0.7481 0s - loss: 0.4880 - accu
Epoch 70/100
1230/1230 [==============================] - 101s 82ms/step - loss: 0.4868 - accuracy: 0.7854 - val_loss: 1.1697 - val_accuracy: 0.7403A - ETA: 48s - l - - - ET - E - ETA: 2s - loss: 0.4882 - accuracy: 0. - ETA:  - ETA: 0s - loss: 0.4863 - accura - ETA: 0s - loss: 0.4867 - accuracy: 
Epoch 71/100
1230/1230 [==============================] - 101s 82ms/step - loss: 0.4877 - accuracy: 0.7845 - val_loss: 0.7355 - val_accuracy: 0.7458
Epoch 72/100
1230/1230 [==============================] - 101s 82ms/step - loss: 0.4862 - accuracy: 0.7860 - val_loss: 0.4182 - val_accuracy: 0.7507
Epoch 73/100
1230/1230 [==============================] - 101s 82ms/step - loss: 0.4863 - accu

1242/1242 [==============================] - 97s 78ms/step - loss: 0.4996 - accuracy: 0.7783 - val_loss: 0.4560 - val_accuracy: 0.7744TA: 35s - loss: 0.5044 - accuracy: - ETA: 34s - loss: 0.5059 - accuracy: 0.772 - ETA: 34s - loss: 0.5062 - - ETA: 33s - l - ETA: 31s - loss: 0.5058 - - ETA: 30s - loss: 0 - ETA: 28s - loss: 0.5063 - accuracy: 0. - ETA: 27s  - ETA: 25 - E
Epoch 11/100
1242/1242 [==============================] - 99s 80ms/step - loss: 0.4975 - accuracy: 0.7796 - val_loss: 0.1911 - val_accuracy: 0.7731
Epoch 12/100
1242/1242 [==============================] - 99s 80ms/step - loss: 0.4978 - accuracy: 0.7789 - val_loss: 0.4527 - val_accuracy: 0.7734
Epoch 13/100
1242/1242 [==============================] - 99s 80ms/step - loss: 0.4972 - accuracy: 0.7791 - val_loss: 0.3882 - val_accuracy: 0.7702
Epoch 14/100
1242/1242 [==============================] - 99s 79ms/step - loss: 0.4962 - accuracy: 0.7790 - val_loss: 0.3949 - val_accuracy: 0.7761
Epoch 15/100
1242/1242 [============

1242/1242 [==============================] - 99s 80ms/step - loss: 0.4923 - accuracy: 0.7813 - val_loss: 0.2848 - val_accuracy: 0.7737accuracy: 0.77 - ETA: 45s - loss: 0.4924 - acc - ETA: 44s - loss: 0.4925 -  - ETA:  - ETA: 40s - loss: 0.4910 - accuracy: 0 - ETA: 39s - loss: 0.4904 - ac - ETA: 38s - loss: 0.4939 - acc - ETA:  - ETA: 35s - loss: 0.5003 - accuracy:  - ETA: 34s - loss: 0.5003 - accuracy: 0 - ETA: 34s - loss: 0.5012 - ETA: 32s -
Epoch 55/100
1242/1242 [==============================] - 99s 80ms/step - loss: 0.4925 - accuracy: 0.7818 - val_loss: 0.3636 - val_accuracy: 0.7757
Epoch 56/100
1242/1242 [==============================] - 100s 81ms/step - loss: 0.4948 - accuracy: 0.7803 - val_loss: 0.1567 - val_accuracy: 0.7725s - loss
Epoch 57/100
1242/1242 [==============================] - 99s 80ms/step - loss: 0.4927 - accuracy: 0.7825 - val_loss: 0.3734 - val_accuracy: 0.7735
Epoch 58/100
1242/1242 [==============================] - 100s 80ms/step - loss: 0.4925 - accuracy: 

1242/1242 [==============================] - 101s 82ms/step - loss: 0.4909 - accuracy: 0.7823 - val_loss: 0.5449 - val_accuracy: 0.77681 - ac - - E
Epoch 99/100
1242/1242 [==============================] - 100s 81ms/step - loss: 0.4917 - accuracy: 0.7819 - val_loss: 0.5017 - val_accuracy: 0.7773
Epoch 100/100
1242/1242 [==============================] - 100s 81ms/step - loss: 0.4920 - accuracy: 0.7814 - val_loss: 0.5336 - val_accuracy: 0.7774
Prediction on test data
best epoch:  005
Counting predicted:  Counter({1: 18264, 0: 1941})

Model Report
Accuracy (test set): 0.7975
Confusion matrix:
[[  954  3105]
 [  987 15159]]
Detailed classification report:
              precision    recall  f1-score   support

           0       0.49      0.24      0.32      4059
           1       0.83      0.94      0.88     16146

    accuracy                           0.80     20205
   macro avg       0.66      0.59      0.60     20205
weighted avg       0.76      0.80      0.77     20205


Model Repor

1235/1235 [==============================] - 100s 81ms/step - loss: 0.4892 - accuracy: 0.7834 - val_loss: 0.8287 - val_accuracy: 0.7812: 35s - loss: 0.4907 - accuracy: 0.780 - ETA: 35s - loss: 0.4909 - accurac - ETA: 34s - loss: 0.4912 - accuracy: 0
Epoch 45/100
1235/1235 [==============================] - 100s 81ms/step - loss: 0.4891 - accuracy: 0.7837 - val_loss: 0.8675 - val_accuracy: 0.7926
Epoch 46/100
1235/1235 [==============================] - 100s 81ms/step - loss: 0.4889 - accuracy: 0.7844 - val_loss: 0.4531 - val_accuracy: 0.7888
Epoch 47/100
1235/1235 [==============================] - 100s 81ms/step - loss: 0.4890 - accuracy: 0.7843 - val_loss: 0.5702 - val_accuracy: 0.7857 l - ETA: 8s - los - ETA: 7s - loss: 0.4918 - accu - ETA: 0s - loss: 0.4887 - 
Epoch 48/100
1235/1235 [==============================] - 99s 80ms/step - loss: 0.4871 - accuracy: 0.7840 - val_loss: 0.3870 - val_accuracy: 0.7889
Epoch 49/100
1235/1235 [==============================] - 100s 81ms/step - lo